In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import re
import nltk

C:\Users\adamg\AppData\Local\Temp\ipykernel_21012\666854454.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# IR Evaluation

In [2]:
relevant_docs = pd.read_csv('qrels.csv')
system_results = pd.read_csv('ttdssystemresults.csv')


In [3]:
def precision(retrieved, relevant):
    retrieved_set = set(retrieved)
    relevant_set = set(relevant)
    return len(retrieved_set.intersection(relevant_set)) / len(retrieved_set)

def recall(retrieved, relevant):
    retrieved_set = set(retrieved)
    relevant_set = set(relevant)
    return len(retrieved_set.intersection(relevant_set)) / len(relevant_set)

def average_precision(retrieved, relevant):
    precision_sum = 0
    for i in range(1, len(retrieved) + 1):
        if retrieved[i - 1] in relevant:
            precision_sum += precision(retrieved[:i], relevant)
    return precision_sum / len(relevant)

def nDCG(retrieved, relevant, relevance_scores):
    scores = {}
    for i in range(len(relevant)):
        scores[relevant[i]] = relevance_scores[i]
    retrieved_scores = []
    for doc in retrieved:
        if doc in scores:
            retrieved_scores.append(scores[doc])
        else:
            retrieved_scores.append(0)

    DCG = retrieved_scores[0]
    for i in range(1, len(retrieved_scores)):
        DCG += retrieved_scores[i] / np.log2(i + 1)
    retrieved_scores.sort(reverse=True)
    IDCG = retrieved_scores[0]
    for i in range(1, len(retrieved_scores)):
        IDCG += retrieved_scores[i] / np.log2(i + 1)
    if IDCG == 0:
        return 0
    return DCG / IDCG


In [4]:
query_ids = system_results['query_number'].unique()
systems = system_results['system_number'].unique()

results = pd.DataFrame(columns=['system_number', 'query_number', 'P@10', 'R@50', 'r-precision', 'AP', 'nDCG@10', 'nDCG@20'])

for system in systems:
    precision_10s = []
    recall_50s = []
    r_precisions = []
    aps = []
    nDCG_10s = []
    nDCG_20s = []
    for query_id in query_ids:
        relevant = relevant_docs[relevant_docs['query_id'] == query_id]['doc_id'].values
        relevance_scores = relevant_docs[relevant_docs['query_id'] == query_id]['relevance'].values
        retrieved = system_results[(system_results['query_number'] == query_id) & (system_results['system_number'] == system)].sort_values(by='rank_of_doc')['doc_number'].values

        precision_10 = precision(retrieved[:10], relevant)
        recall_50 = recall(retrieved[:50], relevant)
        r_precision = precision(retrieved[:len(relevant)], relevant)
        ap = average_precision(retrieved, relevant)
        nDCG_10 = nDCG(retrieved[:10], relevant, relevance_scores)
        nDCG_20 = nDCG(retrieved[:20], relevant, relevance_scores)
        
        results.loc[len(results)] = [int(system), int(query_id), precision_10, recall_50, r_precision, ap, nDCG_10, nDCG_20]

        precision_10s.append(precision_10)
        recall_50s.append(recall_50)
        r_precisions.append(r_precision)
        aps.append(ap)
        nDCG_10s.append(nDCG_10)
        nDCG_20s.append(nDCG_20)

        
    results.loc[len(results)] = [int(system), 'mean', np.mean(precision_10s), np.mean(recall_50s), np.mean(r_precisions), np.mean(aps), np.mean(nDCG_10s), np.mean(nDCG_20s)]


In [5]:
results['system_number'] = results['system_number'].astype(int)
results['query_number'] = results['query_number'].apply(lambda x: 'mean' if x == 'mean' else int(x))
results = results.round(3)
save_results = results.to_csv('ir_eval.csv', index=False)


In [6]:
mean_results = results[results['query_number'] == 'mean']
for column in ['P@10', 'R@50', 'r-precision', 'AP', 'nDCG@10', 'nDCG@20']:
    df = mean_results[['system_number', column]]
    df = df.set_index('system_number')
    df.sort_values(by=column, ascending=False, inplace=True)
    print("Best Model for", column, ":", df.index[0])

    best_model = df.index[0]
    second_best_model = df.index[1]
    best_model_value = df.loc[best_model][column]
    second_best_model_value = df.loc[second_best_model][column]
    # two tailed t-test
    print(best_model, "vs", second_best_model)
    print(best_model_value, "vs", second_best_model_value)
    t_statistic = (best_model_value - second_best_model_value) / np.sqrt((best_model_value * (1 - best_model_value) / 50) + (second_best_model_value * (1 - second_best_model_value) / 50))
    print("t-statistic:", t_statistic)
    print("p-value:", 2 * (1 - stats.t.cdf(t_statistic, 98)))
    print("")

    

Best Model for P@10 : 3
3 vs 5
0.41 vs 0.41
t-statistic: 0.0
p-value: 1.0

Best Model for R@50 : 2
2 vs 1
0.867 vs 0.834
t-statistic: 0.46322462441985235
p-value: 0.6442304510971641

Best Model for r-precision : 3
3 vs 6
0.448 vs 0.448
t-statistic: 0.0
p-value: 1.0

Best Model for AP : 3
3 vs 6
0.451 vs 0.445
t-statistic: 0.060328233925037125
p-value: 0.952017072315338

Best Model for nDCG@10 : 3
3 vs 6
0.592 vs 0.571
t-statistic: 0.21289482573886245
p-value: 0.8318512285899999

Best Model for nDCG@20 : 3
3 vs 1
0.584 vs 0.566
t-statistic: 0.18209000555013286
p-value: 0.8558882168842699



# Text Analysis

In [48]:
bible_and_quran = pd.read_csv("bible_and_quran.tsv", sep='\t', header=None)
bible_and_quran.columns = ['Source', 'Text']
ot, nt, quran = bible_and_quran[bible_and_quran['Source'] == 'OT'], bible_and_quran[bible_and_quran['Source'] == 'NT'], bible_and_quran[bible_and_quran['Source'] == 'Quran']

In [49]:
def tokenize(text):
    """
    Tokenize the text and return a list of words.
    Tokenisation is done by splitting the text, making it lowercase, and replacing any non-alphanumeric characters with spaces.
    :param text: The text to be tokenized
    :return: A list of words
    """
    tokens = text.split()
    words = [re.sub(r'[^a-zA-Z0-9]', ' ', token) for token in tokens]
    words = [word.lower() for word in words if word != '' or word != ' ']
    words = [word.strip() for word in words]
    return words

def remove_stopwords(words, stop_words_file="stop_words.txt"):
    """
    Remove stopwords from the list of words and return the filtered list.
    Stopwords are read from the file specified in the stop_words_file parameter.
    :param words: The list of words
    :param stop_words_file: The path to the file containing the stopwords
    :return: The filtered list of words
    """
    with open(stop_words_file, 'r') as f:
        stop_words = f.readlines()
    stop_words = [word.strip() for word in stop_words]
    filtered_words = [word for word in words if word not in stop_words]
    return filtered_words

def stem(words):
    """
    Stem the words using the Porter stemmer and return the list of stemmed words.
    :param words: The list of words
    :return: The list of stemmed words
    """
    stemmer = nltk.stem.PorterStemmer()
    return [stemmer.stem(word) for word in words]

def preprocess(text):
    """
    Preprocess the text by tokenizing, removing stopwords, and stemming the words.
    :param text: The text to be preprocessed
    :param stopping: A boolean value indicating whether to remove stopwords or not
    :return: The list of preprocessed words
    """
    words = tokenize(text)
    words = remove_stopwords(words)
    words = stem(words)
    return words

In [50]:
ot_tokens = ot['Text'].apply(preprocess)
nt_tokens = nt['Text'].apply(preprocess)
quran_tokens = quran['Text'].apply(preprocess)

In [60]:
# Compute Mutual Information

def mutual_information(word, corpus1, corpus2):
    """
    Compute the mutual information of a word in two corpora.
    :param word: The word for which to compute the mutual information
    :param corpus1: The first corpus
    :param corpus2: The second corpus
    :return: The mutual information of the word in the two corpora
    """

    N_11 = 0
    N_10 = 0
    N_01 = 0
    N_00 = 0

    for doc in corpus1:
        if word in doc:
            N_11 += 1
        else:
            N_01 += 1
    for doc in corpus2:
        if word in doc:
            N_10 += 1
        else:
            N_00 += 1

    N = N_11 + N_10 + N_01 + N_00
    if N_00 == 0 or N_01 == 0 or N_10 == 0 or N_11 == 0:
        return 0

    I = N_11 / N * np.log2(N * N_11 / ((N_10 + N_11) * (N_01 + N_11))) + \
        N_01 / N * np.log2(N * N_01 / ((N_00 + N_01) * (N_01 + N_11))) + \
        N_10 / N * np.log2(N * N_10 / ((N_10 + N_11) * (N_00 + N_10))) + \
        N_00 / N * np.log2(N * N_00 / ((N_00 + N_01) * (N_00 + N_10)))
    
    return I

def top_mutual_information(corpus1, corpus2):
    """
    Compute the top n words with the highest mutual information in two corpora.
    :param corpus1: The first corpus
    :param corpus2: The second corpus
    :param n: The number of words to return
    :return: A list of tuples containing the top n words and their mutual information
    """
    words = set()
    for doc in corpus1:
        new_words = set(doc)
        words = words.union(new_words)
    for doc in corpus2:
        new_words = set(doc)
        words = words.union(new_words)
    print(len(words))
    mi = []
    i = 0
    for word in words:
        mi.append((word, mutual_information(word, corpus1, corpus2)))
        i += 1
        if i % 1000 == 0:
            print(i)

    mi.sort(key=lambda x: x[1], reverse=True)
    return mi

top_mi_ot_quran = top_mutual_information(ot_tokens, quran_tokens)

9003
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [61]:
top_mi_ot_quran

[('god', 0.03720803851237133),
 ('believ', 0.03139173543983355),
 ('israel', 0.027033258872380735),
 ('torment', 0.0256606704080247),
 ('king', 0.024961830154967942),
 ('son', 0.019546477848576535),
 ('revel', 0.0182037526779162),
 ('messeng', 0.01645703428785894),
 ('guidanc', 0.013879797341734272),
 ('deed', 0.013431292190372817),
 ('unjust', 0.013302842270469352),
 ('hous', 0.013218694686628042),
 ('man', 0.012846545243540636),
 ('lord', 0.012655351127247604),
 ('suffer', 0.012201097984429532),
 ('truth', 0.011778512330051127),
 ('creat', 0.011761714644308538),
 ('hell', 0.01162901971136922),
 ('reward', 0.010743728906875426),
 ('reveal', 0.01054125226149931),
 ('evid', 0.010358527187932703),
 ('receiv', 0.010187096548536497),
 ('god ', 0.010127078935888465),
 ('faith', 0.009955100716059758),
 ('citi', 0.009624378936128709),
 ('true', 0.009086284861846969),
 ('david', 0.008697328397544647),
 ('repli', 0.008668447269903855),
 ('jerusalem', 0.00845481830361693),
 ('men', 0.00840253490

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
24816    NaN
24817    NaN
24818    NaN
24819    NaN
24820    NaN
Name: Text, Length: 24821, dtype: object